In [ ]:
!pip install requests
!pip install python-magic


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
import os
from urllib.parse import urlparse
from datetime import datetime
import magic

def get_content_type(url):
    try:
        response = requests.head(url)
        response.raise_for_status()
        content_type = response.headers.get('content-type')
        return content_type.split(';')[0] if content_type else None
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch content type. Error: {e}")
        return None

# Define the URL of the file you want to download
download_url = input("Enter the URL of the file you want to download: ")

# Define the URL of the WebDAV server including the file name
upload_url = ""

# Define the username and password for the WebDAV server
username = ""
password = ""

# Set the chunk size for streaming
chunk_size = 1024 * 1024  # 1 MB

# Check if the WebDAV server is reachable
try:
    response = requests.head(upload_url, auth=HTTPBasicAuth(username, password))
    response.raise_for_status()  # Raise an exception for non-2xx status codes
    print("WebDAV server is reachable. Starting upload...")
except requests.exceptions.RequestException as e:
    print(f"Failed to connect to WebDAV server. Error: {e}")
    exit(1)  # Exit the script if the server is not reachable

# Get the content type of the file
content_type = get_content_type(download_url)
if not content_type:
    print("Failed to determine content type. Please check the URL.")
    exit(1)

# Download the file in chunks and display progress
temp_file_path = "temp_file"
with requests.get(download_url, stream=True) as download_response:
    total_size = int(download_response.headers.get('content-length', 0))
    with tqdm(total=total_size, unit='B', unit_scale=True, desc='Downloading file', ascii=True) as pbar:
        with open(temp_file_path, 'wb') as temp_file:
            for chunk in download_response.iter_content(chunk_size=chunk_size):
                if chunk:
                    temp_file.write(chunk)
                    pbar.update(len(chunk))

# Use python-magic to detect the file type
mime = magic.Magic(mime=True)
file_type = mime.from_file(temp_file_path)

# Modify the upload URL to include a timestamp for uniqueness
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
upload_url = f"{upload_url}file_{timestamp}.{file_type.split('/')[1]}"

# Upload the file in chunks and display progress
with open(temp_file_path, 'rb') as temp_file:
    with tqdm(total=os.path.getsize(temp_file_path), unit='B', unit_scale=True, desc='Uploading file', ascii=True) as pbar:
        temp_file.seek(0)  # Seek back to the beginning of the file
        with requests.put(
            upload_url,
            auth=HTTPBasicAuth(username, password),
            data=temp_file,
            headers={"Content-Type": content_type},
        ) as upload_response:
            if upload_response.status_code == 201:
                print("File uploaded successfully")
            else:
                print(f"Failed to upload file. Status code: {upload_response.status_code}")

# Remove the temporary file
os.remove(temp_file_path)
